# Vector Store

## Produce text on dogs and cats

In [48]:
# Load simple sentences
_texts = [
    "Dogs are great companions, "
    "known for their loyalty and friendliness.",
    "Cats are independent pets that often enjoy their own space.",
]

## Load PDF

In [33]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = "./Rabbits and Guinea Pigs.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

## Setup vector store and retriever tool

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_core.tools import create_retriever_tool

_ollama_embeddings = OllamaEmbeddings(model="nomic-embed-text:v1.5")
_embeddings = OpenAIEmbeddings()

use_ollama_embedding = False
selected_model = _ollama_embeddings if use_ollama_embedding else _embeddings

vector_store = InMemoryVectorStore(_embeddings)

vector_store.add_texts(_texts)
vector_store.add_documents(all_splits)

document_tool = create_retriever_tool(
    vector_store.as_retriever(),
    "pet_information_retriever",
    "Fetches information about pets (dogs, cats, rabbits, and guinea pigs)."
)

## Create agent

In [49]:
from langchain.agents import create_agent
from langchain_ollama.chat_models import ChatOllama
from langchain_openai import ChatOpenAI

ollama_model = ChatOllama(model="mistral:latest")
openai_model = ChatOpenAI(model="gpt-3.5-turbo")

use_ollama = False
selected_model = ollama_model if use_ollama else openai_model

agent = create_agent(
    selected_model,
    tools=[document_tool],
    prompt=f"""You are a helpful pet assistant with access to information about pets.

        Your primary tool is: {document_tool.name}
        Description: {document_tool.description}

        Guidelines:
        - Use the tool to find relevant information about pets (dogs, cats, rabbits, and guinea pigs)
        - Be conversational and friendly in your responses
        - If the user asks about specific pets, characteristics, or care, always search the tool first
        - Provide context from the retrieved information when answering
        - If no relevant information is found, suggest related topics or ask for clarification
        - Keep responses informative but not overwhelming

        Always search for pet-related information before giving general advice.""",
)

In [38]:
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.messages.chat import ChatMessage

result = agent.invoke((
    {"messages": [HumanMessage("What are rabbits known for?")]}
))

In [39]:
import pprint as pp

for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

What are rabbits known for?
================================== Ai Message ==================================
Tool Calls:
  pet_information_retriever (call_Rfwn6fXykYBhOLEH7qgLVlrP)
 Call ID: call_Rfwn6fXykYBhOLEH7qgLVlrP
  Args:
    query: What are rabbits known for?
================================= Tool Message =================================
Name: pet_information_retriever

Rabbits 
Rabbits are gentle, social herbivores that flourish when given space to explore 
combined with secure hiding spots that let them feel sheltered. Their digestive system 
is optimized for near constant intake of high fiber plant matter, so unlimited fresh grass 
hay is the cornerstone of health, keeping gut motility active and naturally wearing down 
continuously growing teeth. Supplemental leafy greens add hydration and 
micronutrients, while calorie dense pellets are best limited to prevent obesity and cecal 
imbalance. S

## For multiple documents

In [40]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="For lunch, I enjoyed a fresh salad with grilled chicken and avocado.",
    metadata={"source": "tweet"},
)

In [41]:
documents = [document_1, document_2]

new_vector_store = InMemoryVectorStore(_embeddings)
new_vector_store.add_documents(documents=documents)

['294b1b6f-2378-4241-8a60-531e41fb7c9f',
 '353f90c4-3d5f-472a-b5c4-ab98f11730a7']

In [42]:
meal_document_tool = create_retriever_tool(
    new_vector_store.as_retriever(),
    "meal_information_retriever",
    "Fetches information about meals and food experiences from tweets."
)

agent = create_agent(
    selected_model,
    tools=[meal_document_tool],
    prompt=f"""You are a helpful food assistant with access to personal meal experiences shared on social media.

        Your primary tool is: {meal_document_tool.name}
        Description: {meal_document_tool.description}

        Guidelines:
        - Use the tool to find relevant information about meals, food experiences, and dining habits from the available tweets
        - Be conversational and friendly in your responses
        - If the user asks about specific meals, ingredients, or food experiences, always search the tool first
        - Provide context from the retrieved information when answering
        - If no relevant information is found, suggest related topics or ask for clarification
        - Keep responses informative but not overwhelming

        Always search for meal-related information before giving general advice.""",
)

In [50]:
test_messages_meal_agent = [
    "What did you have for breakfast?",      # must call tool
    "What did you have for lunch?",          # must call tool
    "What are some meal ideas?",             # maybe call tool
    "Tell me about your food experiences.",  # maybe call tool
    "What's your favorite color?",           # should not call tool
    "How to bake a cake?"                    # should not call tool
]

test_messages_pet_agent = [
    "What are dogs known for?",              # must call tool
    "What are rabbits known for?",           # must call tool
    "Tell me about pets.",                   # maybe call tool
    "What are some pet characteristics?",    # maybe call tool
    "What's the weather today?",             # should not call tool
    "How to train a dog?"                    # should not call tool
]

In [52]:
def test_agent_responses(agent, messages, agent_name="Agent"):
    """Test agent with multiple messages and see tool usage"""
    print(f"\n{'='*50}")
    print(f"Testing {agent_name}")
    print(f"{'='*50}")

    for i, msg in enumerate(messages):
        print(f"\n--- Test {i+1}: '{msg}' ---")
        try:
            result = agent.invoke({"messages": [HumanMessage(msg)]})
            # Check if tool was called
            tool_calls = [m for m in result['messages'] if hasattr(m, 'tool_calls') and m.tool_calls]
            if tool_calls:
                print("✓ Tool was called")
                # Show tool call details
                for call in tool_calls[0].tool_calls:
                    print(f"  Tool: {call['name']}")
            else:
                print("✗ No tool called")

            # Show final response
            final_msg = result['messages'][-1]
            print(f"Response: {final_msg.content[:150]}...")

        except Exception as e:
            print(f"Error: {e}")

test_agent_responses(agent, test_messages_meal_agent, "Meal Agent")
# test_agent_responses(agent, test_messages_pet_agent, "Pet Agent")


Testing Meal Agent

--- Test 1: 'What did you have for breakfast?' ---
✗ No tool called
Response: I'm here to help you with information about pets like dogs, cats, rabbits, and guinea pigs. If you have any questions about pets or need advice on pet...

--- Test 2: 'What did you have for lunch?' ---
✗ No tool called
Response: I'm here to help you with information about pets like dogs, cats, rabbits, and guinea pigs. If you have any questions about pets or need advice on pet...

--- Test 2: 'What did you have for lunch?' ---
✗ No tool called
Response: I'm here to provide information about pets! If you have any questions about dogs, cats, rabbits, or guinea pigs, feel free to ask. How can I assist yo...

--- Test 3: 'What are some meal ideas?' ---
✗ No tool called
Response: I'm here to provide information about pets! If you have any questions about dogs, cats, rabbits, or guinea pigs, feel free to ask. How can I assist yo...

--- Test 3: 'What are some meal ideas?' ---
✓ Tool was called
